In [32]:
import cv2
import csv
import numpy as np

from keras.models import Sequential
from keras.layers import Flatten, Dense, Lambda
from keras.layers.convolutional import Convolution2D
from keras.layers.pooling import MaxPooling2D
from keras.layers import Cropping2D

In [43]:
lines = []
with open("./data/driving_log.csv") as csvfile:
    reader = csv.reader(csvfile)
    for line in reader:
        lines.append(line)
        
images = []
measurements = []
first = False
for line in lines:
    if first == False:
        first = True
        continue
    source_path = line[0]
    filename = source_path.split("/")[-1]
    current_path = "./data/IMG/" + filename
    image = cv2.imread(current_path)
    images.append(image)
    measurement = float(line[3])
    measurements.append(measurement)
    
X_train = np.array(images)
y_train = np.array(measurements)

In [44]:
model = Sequential()
model.add(Lambda(lambda x: x/255.0 - 0.5, input_shape=(160,320,3)))
model.add(Cropping2D(cropping=((70,25), (0,0))))
model.add(Convolution2D(3,(5,5),activation="relu"))
model.add(MaxPooling2D())
model.add(Convolution2D(24,(5,5),activation="relu"))
model.add(MaxPooling2D())
model.add(Convolution2D(36,(5,5),activation="relu"))
model.add(MaxPooling2D())
model.add(Convolution2D(48,(3,3),activation="relu"))
model.add(MaxPooling2D())
#model.add(Convolution2D(64,(3,3),activation="relu"))
#model.add(MaxPooling2D())
model.add(Flatten())
model.add(Dense(1164))
model.add(Dense(100))
model.add(Dense(50))
model.add(Dense(10))
model.add(Dense(1))

model.compile(loss="mse", optimizer="Adam")
model.fit(X_train, y_train, validation_split=0.2, shuffle=True, epochs=1)
model.save("model.h5")

Train on 11304 samples, validate on 2827 samples
Epoch 1/1
11304/11304 [==============================] - 426s - loss: 0.0273 - val_loss: 0.0353
